In [2]:
import os
from dotenv import load_dotenv
load_dotenv() # Load environment variables from a .env file if present

groq_api_key = os.getenv("GROQ_API_KEY")

In [3]:
from langchain_groq import ChatGroq
model=ChatGroq(model="gemma2-9b-it", groq_api_key=groq_api_key)


In [4]:
from langchain_core.messages import HumanMessage
ans=model.invoke([HumanMessage(content="Hii, My name is Yash")])
ans

AIMessage(content="Hi Yash! 👋\n\nIt's nice to meet you. What can I do for you today? 😊\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 15, 'total_tokens': 40, 'completion_time': 0.045454545, 'prompt_time': 0.001251659, 'queue_time': 0.249613411, 'total_time': 0.046706204}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--26d9c648-996c-47f9-9833-f4702107daa8-0', usage_metadata={'input_tokens': 15, 'output_tokens': 25, 'total_tokens': 40})

In [5]:
from langchain_core.messages import AIMessage

model.invoke([
    HumanMessage(content="Hii, My name is Yash"),
    AIMessage(content="Hello Yash! It's nice to meet you. 👋\n\nWhat can I do for you today?\n"),
    HumanMessage(content="can you tell my name?")
])

AIMessage(content='Of course! You said your name is Yash. 😊 \n\nIs there anything else I can help you with?\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 53, 'total_tokens': 79, 'completion_time': 0.047272727, 'prompt_time': 0.001716409, 'queue_time': 0.25286313, 'total_time': 0.048989136}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--9480363c-56ee-41e4-be56-57e37ea7c575-0', usage_metadata={'input_tokens': 53, 'output_tokens': 26, 'total_tokens': 79})

### Message History
We can use a Message History class to wrap our model and make it stateful. This will keep track of inputs and outputs of the model, and store them in some datastore. Future interactions will then load those messages and pass them into the chain as part of the input. Let's see how to use this!

In [6]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store={}

def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]

with_message_history=RunnableWithMessageHistory(model,get_session_history)

In [7]:
config={"configurable":{"session_id":"chat1"}}

In [8]:
response=with_message_history.invoke(
    [HumanMessage(content="Hi , My name is Yash and I am a software developer.")],
    config=config
)

In [9]:
response.content

"Hello Yash! It's nice to meet you. \n\nI'm a large language model, here to help with any questions or tasks you might have.  \n\nWhat kind of software development do you specialize in?\n"

In [10]:
with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

AIMessage(content='Your name is Yash.  I remember that from our introduction! 😊  \n\nIs there anything else I can help you with?\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 29, 'prompt_tokens': 84, 'total_tokens': 113, 'completion_time': 0.052727273, 'prompt_time': 0.002432999, 'queue_time': 0.25076882, 'total_time': 0.055160272}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--8a4924fc-582c-4544-896f-42cabb3d548f-0', usage_metadata={'input_tokens': 84, 'output_tokens': 29, 'total_tokens': 113})

In [11]:
## change the config-->session id
config1={"configurable":{"session_id":"chat2"}}
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

"As an AI, I have no memory of past conversations and do not know your name. If you'd like to tell me, I'd be happy to use it!\n"

In [12]:
response=with_message_history.invoke(
    [HumanMessage(content="Hey My name is John")],
    config=config1
)
response.content

'Hi John! Nice to meet you. 👋 \n\nIs there anything I can help you with today?\n'

In [13]:
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

'Your name is John, as you told me earlier. 😊  \n\nDo you have any other questions for me?\n'

## Prompt Template

In [14]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder


prompt=ChatPromptTemplate.from_messages([
    ("system"," You are a helpful assistant that helps people find information."),
    MessagesPlaceholder(variable_name="messages")
])

chain=prompt|model

In [15]:
chain.invoke({"messages":[HumanMessage(content="Hii, My name is Yash")]})

AIMessage(content="Hi Yash! 👋\n\nIt's nice to meet you. How can I help you today?  \n\nDo you have a question I can answer, or are you looking for information on a specific topic? 😊 \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 48, 'prompt_tokens': 26, 'total_tokens': 74, 'completion_time': 0.087272727, 'prompt_time': 0.001399659, 'queue_time': 0.252118521, 'total_time': 0.088672386}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--938d7226-5cd9-4625-ad79-db11ea7212d0-0', usage_metadata={'input_tokens': 26, 'output_tokens': 48, 'total_tokens': 74})

In [16]:
with_message_history=RunnableWithMessageHistory(chain,get_session_history)

In [17]:
config={"configurable":{"session_id":"chat3"}}
response=with_message_history.invoke(
    [HumanMessage(content="My name is Yash")],
    config=config
)
response.content

"Hello Yash!\n\nIt's nice to meet you. What can I help you find information about today?  Just ask, and I'll do my best to assist you. 😊 \n\n"

In [18]:
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name?")],
    config=config
)
response.content

'Your name is Yash, as you told me when you first introduced yourself.  😊  \n\nIs there anything else I can help you with?\n'

In [19]:
prompt=ChatPromptTemplate.from_messages([
    ("system"," You are a helpful assistant that helps people find information in {language}."),
    MessagesPlaceholder(variable_name="messages")
])

chain=prompt|model

In [20]:
response=chain.invoke(
    {"messages":[HumanMessage(content="Hii, My name is Yash")],
     "language":"Hindi"}
)

response.content

'नमस्ते यश! 👋 \n\nमुझे खुशी है कि आप मुझसे बात कर रहे हैं। आप क्या जानना चाहते हैं? \n\nमैं आपकी मदद करने के लिए तैयार हूँ! बस मुझे बताएं कि क्या जानकारी चाहिए। \n\n'

In [21]:
chain.invoke(
    {"messages":[HumanMessage(content="What is Gen AI?")],"language":"English"}
)

AIMessage(content="Gen AI stands for **Generative Artificial Intelligence**. \n\nIt's a type of artificial intelligence that focuses on creating new content, rather than just analyzing existing data. Think of it like this:\n\n* **Traditional AI:**  Analyzes data to find patterns and make predictions (e.g., recommending a movie you might like).\n* **Gen AI:** Uses learned patterns to generate new content (e.g., writing a poem, composing music, creating an image).\n\n**Here are some examples of what Gen AI can do:**\n\n* **Text Generation:** Writing stories, articles, poems, dialogue, and even code.\n* **Image Generation:** Creating realistic images, artwork, and designs from text descriptions.\n* **Music Generation:** Composing original music in different styles.\n* **Video Generation:** Generating short videos or animating images.\n\n**Some popular Gen AI models include:**\n\n* **GPT-3 (Generative Pre-trained Transformer 3)**: A powerful language model used for text generation.\n* **DA

Let's now wrap this more complicated chain in a Message History class. This time, because there are multiple keys in the input, we need to specify the correct key to use to save the chat history.

In [22]:
with_message_history=RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)

In [23]:
config={"configurable":{"session_id":"chat4"}}

response=with_message_history.invoke(
    {"messages":[HumanMessage(content="My name is Yash")],"language":"Hindi"},
    config=config
)
response.content

'नमस्ते Yash!\n\nमैं आपकी मदद करने के लिए यहाँ हूँ।  मुझे बताइए, आप किस बारे में जानकारी ढूंढ रहे हैं? \n\nमैं आपको हिंदी में बहुत सारे विषयों पर जानकारी दे सकता हूँ।  \n\n'

In [24]:
response=with_message_history.invoke(
    {"messages":[HumanMessage(content="Whats my name?")],"language":"Hindi"},
    config=config
)
response.content

'आपका नाम Yash है। 😊 \n\nमैं याद रखता हूँ!  \n'

### Managing the Conversation History
One important concept to understand when building chatbots is how to manage conversation history. If left unmanaged, the list of messages will grow unbounded and potentially overflow the context window of the LLM. Therefore, it is important to add a step that limits the size of the messages you are passing in.
'trim_messages' helper to reduce how many messages we're sending to the model. The trimmer allows us to specify how many tokens we want to keep, along with other parameters like if we want to always keep the system message and whether to allow partial messages

In [26]:
from langchain_core.messages import SystemMessage, trim_messages

trimmer=trim_messages(
    max_tokens=45,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human"
)

messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]
trimmer.invoke(messages)

[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [28]:
from langchain_core.runnables import RunnablePassthrough
from operator import itemgetter

chain=(
    RunnablePassthrough.assign(messages=itemgetter("messages")|trimmer)
    |prompt|model
)

response=chain.invoke(
    {"messages":messages+[HumanMessage(content="What is my favorite ice cream flavor?")],
     "language":"English"
     }
)

response.content



"As an AI, I don't have access to your personal information, including your favorite ice cream flavor.\n\nWhat's your favorite flavor? 😊  \n"

In [29]:
response=chain.invoke(
    {"messages":messages+[HumanMessage(content="Which math problem i asked?")],
     "language":"English"
     }
)

response.content

'You asked me "What is 2 + 2?".  😊 \n\n\nAnything else I can help you with?\n'

In [32]:
# Lets Wrap it up with message history

with_message_history=RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)

config={"configurable":{"session_id":"chat5"}}

In [34]:
response=with_message_history.invoke(
    {"messages":messages+[HumanMessage(content="What is my Name?")],
     "language":"English"
     },
    config=config
)

response.content

"As an AI, I don't have access to any personal information about you, including your name.  \n\nIf you'd like to tell me your name, I'd be happy to know! 😊  \n"

In [35]:
response = with_message_history.invoke(
    {
        "messages": [HumanMessage(content="what math problem did i ask?")],
        "language": "English",
    },
    config=config,
)

response.content

"As a helpful assistant, I have no memory of past conversations. Every interaction we have is a fresh start. \n\nTo help me understand what math problem you'd like to solve, please ask me! I'm ready for a challenge. 😊  \n\n"